In [1]:
pip install fake-useragent

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install webdriver-manager

     |████████████████████████████████| 127 kB 2.0 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.2
    Uninstalling urllib3-2.2.2:
      Successfully uninstalled urllib3-2.2.2
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

selenium 4.22.0 requires urllib3[socks]<3,>=1.26, but you'll have urllib3 1.25.11 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [61]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from fake_useragent import UserAgent
from tqdm import tqdm

# Настройка WebDriver с изменением User-Agent
ua = UserAgent()
user_agent = ua.random

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Запуск в фоновом режиме
options.add_argument('--disable-gpu')  # Отключение использования GPU
options.add_argument(f'user-agent={user_agent}')  # Установка User-Agent

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Начальная ссылка
base_url = 'https://www.amazon.es/s?i=kitchen&rh=n%3A2165180031&fs=true&page='
page_count = 417  # Количество страниц

# Список для сохранения URL-адресов товаров
product_urls = []

# Проход по всем страницам с отображением шкалы прогресса
for page in tqdm(range(1, page_count + 1), desc="Обработка страниц"):
    user_agent = ua.random
    options.add_argument(f'user-agent={user_agent}')  # Изменение User-Agent для каждой страницы

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    url = f"{base_url}{page}&qid=1721224661&ref=sr_pg_{page}"
    driver.get(url)
    time.sleep(2)  # Время ожидания для загрузки страницы (можно увеличить при необходимости)
    
    # Поиск всех ссылок на товары
    product_elements = driver.find_elements(By.CSS_SELECTOR, 'div.s-main-slot div.s-result-item h2 a')
    for elem in product_elements:
        product_urls.append(elem.get_attribute('href'))
    
    #print(f"Страница {page} обработана")

# Закрытие WebDriver
driver.quit()

# Сохранение URL-адресов в CSV файл
df = pd.DataFrame(product_urls, columns=['Product_URL'])
df.to_csv('testing/amazon_product_urls_4.csv', index=False)

print("Сохранение завершено")


Обработка страниц: 100%|██████████| 417/417 [40:10<00:00,  5.78s/it]

Сохранение завершено


In [68]:
data = pd.read_csv('testing/amazon_product_urls_2.csv')

data.shape

(864, 1)

In [21]:
data.shape

(864, 1)

In [73]:
# Чтение CSV файла с адресами товаров
df = pd.read_csv('testing/amazon_product_urls_2.csv')

# Извлечение URL-адресов товаров в список
product_urls = df['Product_URL'].tolist()

In [75]:
import pandas as pd
from urllib.parse import urlparse, urlunparse, parse_qs, urlencode

# Функция для преобразования URL товара в URL страницы с отзывами
def create_review_url(product_url):
    parsed_url = urlparse(product_url)
    path_parts = parsed_url.path.split('/')
    
    # Проверка, что это правильный URL продукта
    if 'dp' in path_parts:
        product_id = path_parts[path_parts.index('dp') + 1]
        
        review_path = f"/Máquinas-De-Café-Arzum-Ok004-B/product-reviews/{product_id}/ref=cm_cr_dp_d_show_all_btm"
        
        query_params = {'ie': 'UTF8', 'reviewerType': 'all_reviews'}
        
        new_url_parts = (
            parsed_url.scheme,
            parsed_url.netloc,
            review_path,
            parsed_url.params,
            urlencode(query_params),
            parsed_url.fragment
        )
        
        review_url = urlunparse(new_url_parts)
        return review_url
    else:
        return None

# Пример списка URL-адресов товаров

# Список для сохранения URL-адресов страниц с отзывами
review_urls = []

# Преобразование каждого URL-адреса товара в URL-адрес страницы с отзывами
for url in product_urls:
    review_url = create_review_url(url)
    if review_url:
        review_urls.append(review_url)

# Сохранение URL-адресов в CSV файл
df = pd.DataFrame(review_urls, columns=['Review_URL'])
df.to_csv('testing/amazon_reviews_urls.csv', index=False)

print("Сохранение завершено")


Сохранение завершено


In [90]:
# Чтение CSV файла с адресами отзывов
df = pd.read_csv('testing/amazon_reviews_urls.csv')

# Извлечение URL-адресов товаров в список
reviews_urls = df['Review_URL'].tolist()

In [79]:
len(reviews_urls)

864

In [104]:
cleaned_urls = [url.replace('/ref=cm_cr_dp_d_show_all_btm', '') for url in reviews_urls]

In [105]:
cleaned_urls

['https://www.amazon.es/Máquinas-De-Café-Arzum-Ok004-B/product-reviews/B01MZ74PGL?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.es/Máquinas-De-Café-Arzum-Ok004-B/product-reviews/B07STQJFK7?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.es/Máquinas-De-Café-Arzum-Ok004-B/product-reviews/B00T4EZ09E?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.es/Máquinas-De-Café-Arzum-Ok004-B/product-reviews/B07VT5PWNC?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.es/Máquinas-De-Café-Arzum-Ok004-B/product-reviews/B076DR3SW8?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.es/Máquinas-De-Café-Arzum-Ok004-B/product-reviews/B01I1OLX3O?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.es/Máquinas-De-Café-Arzum-Ok004-B/product-reviews/B077J854RD?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.es/Máquinas-De-Café-Arzum-Ok004-B/product-reviews/B005116P70?ie=UTF8&reviewerType=all_reviews',
 'https://www.amazon.es/Máquinas-De-Café-Arzum-Ok004-B/product-r

In [106]:
len(cleaned_urls)

864

In [88]:
len(reviews_urls_test)

25

In [60]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import random
from tqdm import tqdm
from fake_useragent import UserAgent

def get_urls(base_url):
    
    filters = [
        'sortBy=recent',
        'sortBy=helpful',
        'sortBy=rating',
        'filterByStar=one_star',
        'filterByStar=two_star',
        'filterByStar=three_star',
        'filterByStar=four_star',
        'filterByStar=five_star'
    ]
    
    list_urls = []
    #for base_url in reviews_urls:
    #base_url = 'https://www.amazon.es/Máquinas-De-Café-Arzum-Ok004-B/product-reviews/B01MZ74PGL/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1'
    for filter_option in filters:
        for page in range(1, 11):
            # Updating the filter and page number 
            updated_url = f"{base_url}&{filter_option}&pageNumber={page}"
            list_urls.append(updated_url)
    
    print(f'Total number of URLs: {len(list_urls)}')
    return list_urls

def scrape_reviews(urls):
    DRIVER_PATH = '/Users/apple/Downloads/Project_Gnomi_Huekradi/chromedriver'  # Укажите правильный путь к chromedriver
    ua = UserAgent()

    reviews = []
    
    for url in tqdm(urls, desc='Processing URLs', unit='URL'):
        user_agent = ua.random
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument(f"user-agent={user_agent}")

        service = Service(DRIVER_PATH)
        driver = webdriver.Chrome(service=service, options=options)
        
        driver.get(url)
        time.sleep(random.randint(1, 3))

        # Getting reviews
        review_blocks = driver.find_elements(By.CSS_SELECTOR, '.a-section.review')
        for review_block in review_blocks:
            title = review_block.find_element(By.CSS_SELECTOR, '.review-title-content').text.strip()
            rating = review_block.find_element(By.CSS_SELECTOR, '.a-icon-alt').get_attribute('textContent').strip()
            body = review_block.find_element(By.CSS_SELECTOR, '[data-hook="review-body"]').text.strip()
            author = review_block.find_element(By.CSS_SELECTOR, '.a-profile-name').text.strip()
            date = review_block.find_element(By.CSS_SELECTOR, '.review-date').text.strip()

            reviews.append({
                'title': title,
                'rating': rating,
                'body': body,
                'author': author,
                'date': date,
            })

        driver.quit()
    
    print("Done.")
    return reviews

def save_to_csv(reviews, filename):
    df = pd.DataFrame(reviews)
    df.drop_duplicates(subset=['title', 'body', 'author'], inplace=True)
    df.to_csv(filename, index=False, encoding='utf-8')

if __name__ == "__main__":
    start_time = time.perf_counter()  # Засекаем начальное время
    
    i = 0
    #reviews_urls_test
    #for i in range(len(reviews_urls_test)):
    urls = get_urls(reviews_urls_test[i])
        
    #3 iterations
        
    '''reviews_1 = scrape_reviews(urls)
    time.sleep(1)
    reviews_2 = scrape_reviews(urls)
    time.sleep(1)'''
    reviews = scrape_reviews(urls)
        
    #reviews = max(reviews_1, reviews_2, reviews_3, key=len)
    
    save_to_csv(reviews, f'testing/new_reviews_25_items_number_{i}.csv')
        
    #urls = get_urls()
    #reviews = scrape_reviews(urls)
    #save_to_csv(reviews, 'testing/new_reviews_25_items_number.csv')
    
    end_time = time.perf_counter()  # Засекаем конечное время
    
    elapsed_time = end_time - start_time  # Вычисляем разницу во времени
    print(f'Saved {len(reviews)} reviews in testing/new_reviews_filtered_5.csv')
    print(f'Time taken: {elapsed_time:.2f} seconds')


Processing URLs:   0%|          | 0/80 [00:00<?, ?URL/s]

Total number of URLs: 80


Processing URLs: 100%|██████████| 80/80 [04:22<00:00,  3.29s/URL]

Done.
Saved 10 reviews in testing/new_reviews_filtered_5.csv
Time taken: 262.86 seconds


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
import random
from tqdm import tqdm

# List of possible User-Agent strings
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15'
]

def get_urls(base_url):
    #base_url = 'https://www.amazon.es/CREATE-THERA-Cafetera-monodosis-semiautom%C3%A1tica/product-reviews/B0BSH8CYN8?ie=UTF8&reviewerType=avp_only_reviews'
    #base_url = 'https://www.amazon.es/Máquinas-De-Café-Arzum-Ok004-B/product-reviews/B01MZ74PGL?ie=UTF8&reviewerType=all_reviews'
    
    filters = [
        'sortBy=recent',
        'sortBy=helpful',
        'sortBy=rating',
        'filterByStar=one_star',
        'filterByStar=two_star',
        'filterByStar=three_star',
        'filterByStar=four_star',
        'filterByStar=five_star'
    ]
    
    list_urls = []
    for filter_option in filters:
        for page in range(1, 11):
            
            # updating the filter and page number 
            updated_url = f"{base_url}&{filter_option}&pageNumber={page}"
            list_urls.append(updated_url)
    
    return list_urls

def scrape_reviews(urls_jopa):
    reviews = []
    DRIVER_PATH = '/Users/apple/Downloads/Project_Gnomi_Huekradi/chromedriver'

    for url in tqdm(urls_jopa, desc='Processing URLs', unit='URL'): 
        # Randomly select a User-Agent
        user_agent = random.choice(user_agents)
        
        # Set up Chrome options
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument(f"user-agent={user_agent}")

        service = Service(DRIVER_PATH)
        driver = webdriver.Chrome(service=service, options=options)
        
        driver.get(url)
        
        time.sleep(random.randint(1, 3))

        # Getting reviews
        review_blocks = driver.find_elements(By.CSS_SELECTOR, '.a-section.review')
        for review_block in review_blocks:
            title = review_block.find_element(By.CSS_SELECTOR, '.review-title-content').text.strip()
            rating = review_block.find_element(By.CSS_SELECTOR, '.a-icon-alt').get_attribute('textContent').strip()
            body = review_block.find_element(By.CSS_SELECTOR, '[data-hook="review-body"]').text.strip()
            author = review_block.find_element(By.CSS_SELECTOR, '.a-profile-name').text.strip()
            date = review_block.find_element(By.CSS_SELECTOR, '.review-date').text.strip()

            reviews.append({
                'title': title,
                'rating': rating,
                'body': body,
                'author': author,
                'date': date,
            })
        
        driver.quit()
    
    print("Done.")
    return reviews    

# Removing duplicates and saving data
def save_to_csv(reviews, filename):
    df = pd.DataFrame(reviews)
    df.drop_duplicates(subset=['title', 'body', 'author'], inplace=True)
    df.to_csv(filename, index=False, encoding='utf-8')

if __name__ == "__main__":
    start_time = time.perf_counter() 
    
    #reviews_urls_test
    for i in range(25, len(cleaned_urls)):
        urls = get_urls(cleaned_urls[i])

        #3 iterations
        
        reviews = []
        reviews = scrape_reviews(urls)
        print(f'Got {len(reviews)} reviews from {cleaned_urls[0]}')
    
        save_to_csv(reviews, f'testing/0-{i:06}.csv')
        print(f'Saved {len(reviews)} reviews in testing/0-{i:06}.csv')
        
    end_time = time.perf_counter()
    elapsed_time = end_time - start_time  # Вычисляем разницу во времени
    print(f'Time taken: {elapsed_time:.2f} seconds')

In [122]:
def save_to_csv(reviews, filename):
    df = pd.DataFrame(reviews)
   # df.drop_duplicates(subset=['title', 'body', 'author'], inplace=True)
    df.to_csv(filename, index=False, encoding='utf-8')

In [123]:
import os
testing_dataframes = []
for j in range(185):
    
    
    file_path = os.path.join('', f'database/reviews/0-{j:06}.csv')
    # Чтение CSV файла и добавление его в список
    try:
        a = pd.read_csv(file_path)
        testing_dataframes.append(a)

    except:
        print(f'No data in testing/0-{j:06}.csv')
# Объединение всех DataFrame в один

#a = pd.read_csv('normal/reviews_filtered.csv')
#testing_dataframes.append(a)

testing_combined_df = pd.concat(testing_dataframes, ignore_index=True)
testing_combined_df.drop_duplicates(inplace=True)

No data in testing/0-000013.csv
No data in testing/0-000022.csv
No data in testing/0-000033.csv
No data in testing/0-000039.csv
No data in testing/0-000045.csv
No data in testing/0-000046.csv
No data in testing/0-000055.csv
No data in testing/0-000056.csv
No data in testing/0-000057.csv
No data in testing/0-000058.csv
No data in testing/0-000059.csv
No data in testing/0-000060.csv
No data in testing/0-000061.csv
No data in testing/0-000062.csv
No data in testing/0-000063.csv
No data in testing/0-000064.csv
No data in testing/0-000065.csv
No data in testing/0-000066.csv
No data in testing/0-000067.csv
No data in testing/0-000068.csv
No data in testing/0-000069.csv
No data in testing/0-000070.csv
No data in testing/0-000071.csv
No data in testing/0-000072.csv
No data in testing/0-000073.csv
No data in testing/0-000074.csv
No data in testing/0-000075.csv
No data in testing/0-000076.csv
No data in testing/0-000077.csv
No data in testing/0-000078.csv
No data in testing/0-000079.csv
No data 

In [124]:
testing_combined_df.shape

(1876, 5)

In [125]:
save_to_csv(testing_dataframes, 'normal/out.csv')